In [7]:
#Import modules
import numpy as np
import pandas as pd
import os, sys
import time as t
import matplotlib as mpl
#mpl.use('Agg')
import pathlib
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib.ticker import MaxNLocator

In [14]:
#CONSTANTS
cwd_PYTHON = os.getcwd() + '/'
PERIOD = 0.1
RSMALL = 0.15
RLARGE = 2.0*RSMALL
EPSILON = 0.4
AMPLITUDE = EPSILON*RLARGE
DT = 1.0e-4
DENSITY = 2.0
MASS_LARGE = DENSITY*4.0*np.pi*RLARGE**3
MASS_SMALL = DENSITY*4.0*np.pi*RSMALL**3
OMEGA = 2.0*np.pi/PERIOD

SVAL_LIST = [1.0,2.0,3.0,4.0,5.0,7.5,10.0,50.0,100.0]
csfont = {'fontname':'Times New Roman'}

In [9]:
#GET FILE NAMES
# constructs a filepath for the pos data of s = $sval
def pname(cwd,eps,sval):
    strDir = cwd+'../../PosData/eps{0}/'.format(eps)
    return strDir+"pd_e{0}_s{1}.txt".format(eps,sval)

# constructs a filepath for the force data of s = $sval
def fname(cwd,eps,sval):
    strDir = cwd+'../../ForceData/eps{0}/'.format(eps)
    return strDir+"fd_s{0}.txt".format(sval)

def plotName(cwd,eps,sval):
    #field = Vort, Pres, Vel, AvgW, AvgP, AvgU
    strDir = cwd+"../../Figures/eps{0}/Forces/".format(eps)
    pathlib.Path(strDir).mkdir(parents=True, exist_ok=True)
    return strDir+"Fhat_s{0}.png".format(sval)


In [10]:
%matplotlib inline
#AUXILIARY FUNCTIONS
#DERIVATIVES
def CentralDifference(data,deriv,var,dt):
    #Perform a central differencing to obtain velocities and accelerations
    for idx in range(1,len(data['time'])-1):
        data.loc[idx,deriv] = (data.loc[idx+1,var] - data.loc[idx-1,var])/(2.0*dt)
    return data[deriv]

#INTEGRALS
def Trapezoidal(data, minBound, maxBound, nTime):
    h = (maxBound-minBound)/float(nTime)
    #print(h)
    s = 0.5*(data[0] + data[nTime-1])
    for idx in range(1,nTime-1):
        s = s + data[idx]
    return h*s

#PLOTTING
def PlotFhat(cwd,time,Fnet,Fapp,Fdrag,eps,sval):
    fig, ax = plt.subplots(nrows=3, ncols=2,figsize=(12,6),dpi=200)
    #Titles and Labels
    ax[0,0].set_title(r'$\hat{F_x}$ vs. #Per: $s^2$ = %.1f'%sval,fontsize=16,**csfont)
    ax[0,1].set_title(r'$\hat{F_y}$ vs. #Per: $s^2$ = %.1f'%sval,fontsize=16,**csfont)
    ax[2,0].set_xlabel('time (nPer)',fontsize=12,**csfont)
    ax[2,1].set_xlabel('time (nPer)',fontsize=12,**csfont)
    ax[0,0].set_ylabel(r'$\hat{F_xc}$ (N)',fontsize=12,**csfont)
    ax[0,1].set_ylabel(r'$\hat{F_yc}$ (N)',fontsize=12,**csfont)
    ax[1,0].set_ylabel(r'$\hat{F_xs}$ (N)',fontsize=12,**csfont)
    ax[1,1].set_ylabel(r'$\hat{F_ys}$ (N)',fontsize=12,**csfont)
    ax[2,0].set_ylabel(r'|$\hat{F_x}$| (N)',fontsize=12,**csfont)
    ax[2,1].set_ylabel(r'|$\hat{F_y}$| (N)',fontsize=12,**csfont)
    #Plot F_hats
    #Large sphere
    #cosine
    ax[0,0].plot(time,Fnet[:,0,0],label='L: net',c='k')
    ax[0,0].plot(time,Fapp[:,0,0],label='L: app',c='r')
    ax[0,0].plot(time,Fdrag[:,0,0],label='L: drag',c='b')
    ax[0,1].plot(time,Fnet[:,2,0],label='L: net',c='k')
    ax[0,1].plot(time,Fapp[:,2,0],label='L: app',c='r')
    ax[0,1].plot(time,Fdrag[:,2,0],label='L: drag',c='b')
    #sine
    ax[1,0].plot(time,Fnet[:,1,0],label='L: net',c='k')
    ax[1,0].plot(time,Fapp[:,1,0],label='L: app',c='r')
    ax[1,0].plot(time,Fdrag[:,1,0],label='L: drag',c='b')
    ax[1,1].plot(time,Fnet[:,3,0],label='L: net',c='k')
    ax[1,1].plot(time,Fapp[:,3,0],label='L: app',c='r')
    ax[1,1].plot(time,Fdrag[:,3,0],label='L: drag',c='b')
    #Mag
    magFxnetL = np.hypot(Fnet[:,0,0],Fnet[:,1,0])
    magFynetL = np.hypot(Fnet[:,2,0],Fnet[:,3,0])
    magFxappL = np.hypot(Fapp[:,0,0],Fapp[:,1,0])
    magFyappL = np.hypot(Fapp[:,2,0],Fapp[:,3,0])
    magFxdragL = np.hypot(Fdrag[:,0,0],Fdrag[:,1,0])
    magFydragL = np.hypot(Fdrag[:,2,0],Fdrag[:,3,0])
    ax[2,0].plot(time,magFxnetL,label='L: net',c='k')
    ax[2,0].plot(time,magFxappL,label='L: app',c='r')
    ax[2,0].plot(time,magFxdragL,label='L: drag',c='b')
    ax[2,1].plot(time,magFynetL,label='L: net',c='k')
    ax[2,1].plot(time,magFyappL,label='L: app',c='r')
    ax[2,1].plot(time,magFydragL,label='L: drag',c='b')
    
    #Small sphere
    #cosine
    ax[0,0].plot(time,Fnet[:,0,1],label='S: net',c='k',ls='--')
    ax[0,0].plot(time,Fapp[:,0,1],label='S: app',c='r',ls='--')
    ax[0,0].plot(time,Fdrag[:,0,1],label='S: drag',c='b',ls='--')
    ax[0,1].plot(time,Fnet[:,2,1],label='S: net',c='k',ls='--')
    ax[0,1].plot(time,Fapp[:,2,1],label='S: app',c='r',ls='--')
    ax[0,1].plot(time,Fdrag[:,2,1],label='S: drag',c='b',ls='--')
    #sine
    ax[1,0].plot(time,Fnet[:,1,1],label='S: net',c='k',ls='--')
    ax[1,0].plot(time,Fapp[:,1,1],label='S: app',c='r',ls='--')
    ax[1,0].plot(time,Fdrag[:,1,1],label='S: drag',c='b',ls='--')
    ax[1,1].plot(time,Fnet[:,3,1],label='S: net',c='k',ls='--')
    ax[1,1].plot(time,Fapp[:,3,1],label='S: app',c='r',ls='--')
    ax[1,1].plot(time,Fdrag[:,3,1],label='S: drag',c='b',ls='--')
    #Mag
    magFxnetS = np.hypot(Fnet[:,0,1],Fnet[:,1,1])
    magFynetS = np.hypot(Fnet[:,2,1],Fnet[:,3,1])
    magFxappS = np.hypot(Fapp[:,0,1],Fapp[:,1,1])
    magFyappS = np.hypot(Fapp[:,2,1],Fapp[:,3,1])
    magFxdragS = np.hypot(Fdrag[:,0,1],Fdrag[:,1,1])
    magFydragS = np.hypot(Fdrag[:,2,1],Fdrag[:,3,1])
    ax[2,0].plot(time,magFxnetS,label='S: net',c='k',ls='--')
    ax[2,0].plot(time,magFxappS,label='S: app',c='r',ls='--')
    ax[2,0].plot(time,magFxdragS,label='S: drag',c='b',ls='--')
    ax[2,1].plot(time,magFynetS,label='S: net',c='k',ls='--')
    ax[2,1].plot(time,magFyappS,label='S: app',c='r',ls='--')
    ax[2,1].plot(time,magFydragS,label='S: drag',c='b',ls='--')
    
    ax[0,0].legend(fontsize='x-small',loc='best')
    ax[0,1].legend(fontsize='x-small',loc='best')
    fig.tight_layout()
    fig.savefig(plotName(cwd,eps,sval))
    fig.clf()
    plt.close()
    
    return


In [11]:
def GetData(cwd,eps,sval):
    
    #Obtain Pos Data
    #xL yL xS yS d_curr d_des time
    posArr = np.loadtxt(pname(cwd,eps,sval),delimiter=' ')
    posDict = {'xL':posArr[:,0],'yL':posArr[:,1],'xS':posArr[:,2],'yS':posArr[:,3],'time':posArr[:,6]}
    posData = pd.DataFrame(data=posDict)
    #Add initial positioning
    initDict = {'xL':[-0.00158344],'yL':[0.45],'xS':[-0.00159739],'yS':[-0.45],'time':[0.0]}
    initData = pd.DataFrame(data=initDict)
    posData = posData.append(initData)
    posData = posData.sort_values(by=['time'])
    posData = posData.reset_index(drop=True)
    
    #Calculate Instantaneous Velocity (Central Difference)
    #Large Sphere
    posData['vxL'], posData['vyL'] = 0.0, 0.0
    posData['axL'], posData['ayL'] = 0.0, 0.0
    posData['vxL'] = CentralDifference(posData,'vxL','xL',dt=DT)
    posData['vyL'] = CentralDifference(posData,'vyL','yL',dt=DT)
    posData['axL'] = CentralDifference(posData,'axL','vxL',dt=DT)
    posData['ayL'] = CentralDifference(posData,'ayL','vyL',dt=DT)
    #Small Sphere
    posData['vxS'], posData['vyS'] = 0.0, 0.0
    posData['axS'], posData['ayS'] = 0.0, 0.0
    posData['vxS'] = CentralDifference(posData,'vxS','xS',dt=DT)
    posData['vyS'] = CentralDifference(posData,'vyS','yS',dt=DT)
    posData['axS'] = CentralDifference(posData,'axS','vxS',dt=DT)
    posData['ayS'] = CentralDifference(posData,'ayS','vyS',dt=DT)
    
    #Save VelData
    velDict = {'vxL':posData['vxL'],'vyL':posData['vyL'],'vxS':posData['vxS'],'vyS':posData['vyS'],'time':posData['time']}
    velData = pd.DataFrame(data=velDict)
    #I Might need to do something more with this data
    
    #Save AccData
    accDict = {'axL':posData['axL'],'ayL':posData['ayL'],'axS':posData['axS'],'ayS':posData['ayS'],'time':posData['time']}
    accData = pd.DataFrame(data=accDict)
    
    #Create Force dataframe
    #Add Net force on each sphere
    forceDict = {'fnxL':accData['axL']*MASS_LARGE,'fnyL':accData['ayL']*MASS_LARGE,
                 'fnxS':accData['axS']*MASS_SMALL,'fnyS':accData['ayS']*MASS_SMALL,'time':accData['time']}
    forceData = pd.DataFrame(data=forceDict)
    #Obtain Applied force data (Spring force data)
    springForceData = np.loadtxt(fname(cwd,eps,sval),delimiter=' ')
    print(springForceData[:,:10])
    print(springForceData.shape)
    #Add spring forces to force dataframe
    #start = np.array([[0.0],[0.0],[0.0],[0.0],[0.0],[0.0],[0.0]])
    
    springForceData = np.insert(springForceData,0,0.0,axis=0)
    #springForceData = np.append(springForceData,np.zeros(7).T)
    print(springForceData[:,:10])
    print(springForceData.shape)
    #springForceData = np.sort(springForceData, axis=6)
    forceData['fsxL'] = springForceData[:,0].copy()
    forceData['fsyL'] = springForceData[:,1].copy()
    forceData['fsxS'] = springForceData[:,3].copy()
    forceData['fsyS'] = springForceData[:,4].copy()
    print(forceData.head())
    
    #Calculate Drage Forces for each sphere
    #Large sphere
    forceData['fdxL'] = forceData['fnxL'] - forceData['fsxL']
    forceData['fdyL'] = forceData['fnyL'] - forceData['fsyL']
    #Small sphere
    forceData['fdxS'] = forceData['fnxS'] - forceData['fsxS']
    forceData['fdyS'] = forceData['fnyS'] - forceData['fsyS']
    
    #Calculate cosine and i*sine terms for each force
    #Large sphere
    #Net
    forceData['fnxLc'] = forceData['fnxL']*np.cos(2.0*forceData['time']*np.pi/PERIOD)
    forceData['fnxLs'] = forceData['fnxL']*np.sin(2.0*forceData['time']*np.pi/PERIOD)
    forceData['fnyLc'] = forceData['fnyL']*np.cos(2.0*forceData['time']*np.pi/PERIOD)
    forceData['fnyLs'] = forceData['fnyL']*np.sin(2.0*forceData['time']*np.pi/PERIOD)
    #Spring
    forceData['fsxLc'] = forceData['fsxL']*np.cos(2.0*forceData['time']*np.pi/PERIOD)
    forceData['fsxLs'] = forceData['fsxL']*np.sin(2.0*forceData['time']*np.pi/PERIOD)
    forceData['fsyLc'] = forceData['fsyL']*np.cos(2.0*forceData['time']*np.pi/PERIOD)
    forceData['fsyLs'] = forceData['fsyL']*np.sin(2.0*forceData['time']*np.pi/PERIOD)
    #Drag
    forceData['fdxLc'] = forceData['fdxL']*np.cos(2.0*forceData['time']*np.pi/PERIOD)
    forceData['fdxLs'] = forceData['fdxL']*np.sin(2.0*forceData['time']*np.pi/PERIOD)
    forceData['fdyLc'] = forceData['fdyL']*np.cos(2.0*forceData['time']*np.pi/PERIOD)
    forceData['fdyLs'] = forceData['fdyL']*np.sin(2.0*forceData['time']*np.pi/PERIOD)
    #Small sphere
    #Net
    forceData['fnxSc'] = forceData['fnxS']*np.cos(2.0*forceData['time']*np.pi/PERIOD)
    forceData['fnxSs'] = forceData['fnxS']*np.sin(2.0*forceData['time']*np.pi/PERIOD)
    forceData['fnySc'] = forceData['fnyS']*np.cos(2.0*forceData['time']*np.pi/PERIOD)
    forceData['fnySs'] = forceData['fnyS']*np.sin(2.0*forceData['time']*np.pi/PERIOD)
    #Spring
    forceData['fsxSc'] = forceData['fsxS']*np.cos(2.0*forceData['time']*np.pi/PERIOD)
    forceData['fsxSs'] = forceData['fsxS']*np.sin(2.0*forceData['time']*np.pi/PERIOD)
    forceData['fsySc'] = forceData['fsyS']*np.cos(2.0*forceData['time']*np.pi/PERIOD)
    forceData['fsySs'] = forceData['fsyS']*np.sin(2.0*forceData['time']*np.pi/PERIOD)
    #Drag
    forceData['fdxSc'] = forceData['fdxS']*np.cos(2.0*forceData['time']*np.pi/PERIOD)
    forceData['fdxSs'] = forceData['fdxS']*np.sin(2.0*forceData['time']*np.pi/PERIOD)
    forceData['fdySc'] = forceData['fdyS']*np.cos(2.0*forceData['time']*np.pi/PERIOD)
    forceData['fdySs'] = forceData['fdyS']*np.sin(2.0*forceData['time']*np.pi/PERIOD)
    
    #Using force data, calculate the hat terms for each force (net, spring, drag)
    nSteps_per = int(PERIOD/DT)
    nPer = int(np.trunc(len(forceData['time'])/nSteps_per))
    #i: per num
    #j: 0 = net; 1 = spring; 2 = drag
    #k: 0 = xcos; 1 = x(isin); 2 = ycos; 3 = y(isin)
    #l: 0 = L; 1 = S
    hatFnet = np.zeros((nPer,4,2))
    hatFapp = np.zeros((nPer,4,2))
    hatFdrag = np.zeros((nPer,4,2))
    timeData = np.zeros(nPer)
    #For every period
    for idx in range(nPer):
        start_idx = idx*nSteps_per
        end_idx = start_idx + nSteps_per
        #Record time
        timeData[idx] = float(idx+1)
        #Large Sphere
        #Net
        hatFnet[idx,0,0] = (1.0/PERIOD)*Trapezoidal(forceData.loc[start_idx:end_idx,'fnxLc'].values.tolist(),0,PERIOD,nSteps_per)
        hatFnet[idx,1,0] = (1.0/PERIOD)*Trapezoidal(forceData.loc[start_idx:end_idx,'fnxLs'].values.tolist(),0,PERIOD,nSteps_per)
        hatFnet[idx,2,0] = (1.0/PERIOD)*Trapezoidal(forceData.loc[start_idx:end_idx,'fnyLc'].values.tolist(),0,PERIOD,nSteps_per)
        hatFnet[idx,3,0] = (1.0/PERIOD)*Trapezoidal(forceData.loc[start_idx:end_idx,'fnyLs'].values.tolist(),0,PERIOD,nSteps_per)
        #Spring
        hatFapp[idx,0,0] = (1.0/PERIOD)*Trapezoidal(forceData.loc[start_idx:end_idx,'fsxLc'].values.tolist(),0,PERIOD,nSteps_per)
        hatFapp[idx,1,0] = (1.0/PERIOD)*Trapezoidal(forceData.loc[start_idx:end_idx,'fsxLs'].values.tolist(),0,PERIOD,nSteps_per)
        hatFapp[idx,2,0] = (1.0/PERIOD)*Trapezoidal(forceData.loc[start_idx:end_idx,'fsyLc'].values.tolist(),0,PERIOD,nSteps_per)
        hatFapp[idx,3,0] = (1.0/PERIOD)*Trapezoidal(forceData.loc[start_idx:end_idx,'fsyLs'].values.tolist(),0,PERIOD,nSteps_per)
        #Drag
        hatFdrag[idx,0,0] = (1.0/PERIOD)*Trapezoidal(forceData.loc[start_idx:end_idx,'fdxLc'].values.tolist(),0,PERIOD,nSteps_per)
        hatFdrag[idx,1,0] = (1.0/PERIOD)*Trapezoidal(forceData.loc[start_idx:end_idx,'fdxLs'].values.tolist(),0,PERIOD,nSteps_per)
        hatFdrag[idx,2,0] = (1.0/PERIOD)*Trapezoidal(forceData.loc[start_idx:end_idx,'fdyLc'].values.tolist(),0,PERIOD,nSteps_per)
        hatFdrag[idx,3,0] = (1.0/PERIOD)*Trapezoidal(forceData.loc[start_idx:end_idx,'fdyLs'].values.tolist(),0,PERIOD,nSteps_per)
        #Small Sphere
        #Net
        hatFnet[idx,0,1] = (1.0/PERIOD)*Trapezoidal(forceData.loc[start_idx:end_idx,'fnxSc'].values.tolist(),0,PERIOD,nSteps_per)
        hatFnet[idx,1,1] = (1.0/PERIOD)*Trapezoidal(forceData.loc[start_idx:end_idx,'fnxSs'].values.tolist(),0,PERIOD,nSteps_per)
        hatFnet[idx,2,1] = (1.0/PERIOD)*Trapezoidal(forceData.loc[start_idx:end_idx,'fnySc'].values.tolist(),0,PERIOD,nSteps_per)
        hatFnet[idx,3,1] = (1.0/PERIOD)*Trapezoidal(forceData.loc[start_idx:end_idx,'fnySs'].values.tolist(),0,PERIOD,nSteps_per)
        #Spring
        hatFapp[idx,0,1] = (1.0/PERIOD)*Trapezoidal(forceData.loc[start_idx:end_idx,'fsxSc'].values.tolist(),0,PERIOD,nSteps_per)
        hatFapp[idx,1,1] = (1.0/PERIOD)*Trapezoidal(forceData.loc[start_idx:end_idx,'fsxSs'].values.tolist(),0,PERIOD,nSteps_per)
        hatFapp[idx,2,1] = (1.0/PERIOD)*Trapezoidal(forceData.loc[start_idx:end_idx,'fsySc'].values.tolist(),0,PERIOD,nSteps_per)
        hatFapp[idx,3,1] = (1.0/PERIOD)*Trapezoidal(forceData.loc[start_idx:end_idx,'fsySs'].values.tolist(),0,PERIOD,nSteps_per)
        #Drag
        hatFdrag[idx,0,1] = (1.0/PERIOD)*Trapezoidal(forceData.loc[start_idx:end_idx,'fdxSc'].values.tolist(),0,PERIOD,nSteps_per)
        hatFdrag[idx,1,1] = (1.0/PERIOD)*Trapezoidal(forceData.loc[start_idx:end_idx,'fdxSs'].values.tolist(),0,PERIOD,nSteps_per)
        hatFdrag[idx,2,1] = (1.0/PERIOD)*Trapezoidal(forceData.loc[start_idx:end_idx,'fdySc'].values.tolist(),0,PERIOD,nSteps_per)
        hatFdrag[idx,3,1] = (1.0/PERIOD)*Trapezoidal(forceData.loc[start_idx:end_idx,'fdySs'].values.tolist(),0,PERIOD,nSteps_per) 
    
    return (velData, timeData, hatFnet, hatFapp, hatFdrag)

In [16]:
#Plan of Attack
#1) For each sval, obtain pos data and force data
#2) Calculate instantaneous velocity and accelerations
#3) From acceleration, calculate net force
#4) The force data given is F_app. F_net = F_drag + F_app
#5) F_drag = F_net - F_app
#6) Given all 3 Forces, calculate the hat versions for each and for each sphere for each period

dataDict = {'s2':[],'epsilon':[],'a':[],'omega':[],'rho':[],'fd1x_real':[],'fd1x_imag':[],'fd1y_real':[],'fd1y_imag':[],
            'fd2x_real':[],'fdx2_imag':[],'fd2y_real':[],'fd2y_imag':[]}
allData = pd.DataFrame(data=dataDict)

#Parameters
eps = EPSILON

for sval in SVAL_LIST:
    mu = OMEGA*DENSITY*RLARGE**2/(2.0*sval)
    velData, timeData, hatFnet, hatFapp, hatFdrag = GetData(cwd_PYTHON,eps,sval)
    #hatFnet = hatFnet/(RLARGE**2*mu*OMEGA)
    #hatFapp = hatFapp/(RLARGE**2*mu*OMEGA)
    #hatFdrag = hatFdrag/(RLARGE**2*mu*OMEGA)
    PlotFhat(cwd_PYTHON,timeData,hatFnet,hatFapp,hatFdrag,eps,sval)
    simDict = {'s2':[sval],'epsilon':[eps],'a':[RLARGE],'omega':[OMEGA],'rho':[DENSITY],
               'fd1x_real':[hatFdrag[-1,0,0]],'fd1x_imag':[hatFdrag[-1,1,0]],
               'fd1y_real':[hatFdrag[-1,2,0]],'fd1y_imag':[hatFdrag[-1,3,0]],
               'fd2x_real':[hatFdrag[-1,0,1]],'fd2x_imag':[hatFdrag[-1,1,1]],
               'fd2y_real':[hatFdrag[-1,2,1]],'fd2y_imag':[hatFdrag[-1,3,1]],
               'fn1x_real':[hatFnet[-1,0,0]],'fn1x_imag':[hatFnet[-1,1,0]],
               'fn1y_real':[hatFnet[-1,2,0]],'fn1y_imag':[hatFnet[-1,3,0]],
               'fn2x_real':[hatFnet[-1,0,1]],'fn2x_imag':[hatFnet[-1,1,1]],
               'fn2y_real':[hatFnet[-1,2,1]],'fn2y_imag':[hatFnet[-1,3,1]],
               'fs1x_real':[hatFapp[-1,0,0]],'fs1x_imag':[hatFapp[-1,1,0]],
               'fs1y_real':[hatFapp[-1,2,0]],'fs1y_imag':[hatFapp[-1,3,0]],
               'fs2x_real':[hatFapp[-1,0,1]],'fs2x_imag':[hatFapp[-1,1,1]],
               'fs2y_real':[hatFapp[-1,2,1]],'fs2y_imag':[hatFapp[-1,3,1]]}
    simData = pd.DataFrame(data=simDict)
    allData = pd.concat([allData,simData],ignore_index=True)
    
allData = allData.sort_values(by=['s2'])
#allData.to_csv(cwd_PYTHON+'hatDragData_e{0}.csv'.format(eps),index=False,sep=' ',float_format='%.5e')
allData.to_csv(cwd_PYTHON+'hatDragData_e{0}_dimensional.csv'.format(eps),index=False,sep=' ',float_format='%.5e')


[[ 3.96079e+01  7.63701e+01 -3.76863e+01 ... -7.63701e+01  3.46391e+00
   1.00000e-04]
 [ 3.95860e+01  9.49012e+01 -3.76646e+01 ... -9.49012e+01  3.45061e+00
   2.00000e-04]
 [ 3.95445e+01  1.13089e+02 -3.76237e+01 ... -1.13089e+02  3.42608e+00
   3.00000e-04]
 ...
 [-5.66067e-02  6.92469e+01  3.13211e-02 ... -6.92469e+01  1.51143e-02
   1.57600e+00]
 [-5.61368e-02  7.03709e+01  3.05331e-02 ... -7.03709e+01  1.53695e-02
   1.57610e+00]
 [-5.56603e-02  7.14924e+01  2.97417e-02 ... -7.14924e+01  1.56245e-02
   1.57620e+00]]
(15762, 7)
[[ 0.00000e+00  0.00000e+00  0.00000e+00 ...  0.00000e+00  0.00000e+00
   0.00000e+00]
 [ 3.96079e+01  7.63701e+01 -3.76863e+01 ... -7.63701e+01  3.46391e+00
   1.00000e-04]
 [ 3.95860e+01  9.49012e+01 -3.76646e+01 ... -9.49012e+01  3.45061e+00
   2.00000e-04]
 ...
 [-5.66067e-02  6.92469e+01  3.13211e-02 ... -6.92469e+01  1.51143e-02
   1.57600e+00]
 [-5.61368e-02  7.03709e+01  3.05331e-02 ... -7.03709e+01  1.53695e-02
   1.57610e+00]
 [-5.56603e-02  7.149

/Users/thomas/anaconda3/envs/tf-research/lib/python3.6/site-packages/ipykernel_launcher.py:37: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



[[ 3.96079e+01  7.63701e+01 -3.76863e+01 ... -7.63701e+01  3.46391e+00
   1.00000e-04]
 [ 3.95852e+01  9.48854e+01 -3.76639e+01 ... -9.48854e+01  3.45005e+00
   2.00000e-04]
 [ 3.95417e+01  1.13023e+02 -3.76209e+01 ... -1.13023e+02  3.42367e+00
   3.00000e-04]
 ...
 [-3.47787e-05  5.43907e+01 -5.84241e-03 ... -5.43907e+01  1.53169e-02
   3.50820e+00]
 [-1.48180e-04  5.37585e+01 -5.70406e-03 ... -5.37585e+01  1.51716e-02
   3.50830e+00]
 [-2.59909e-04  5.31243e+01 -5.56850e-03 ... -5.31243e+01  1.50271e-02
   3.50840e+00]]
(35084, 7)
[[ 0.00000e+00  0.00000e+00  0.00000e+00 ...  0.00000e+00  0.00000e+00
   0.00000e+00]
 [ 3.96079e+01  7.63701e+01 -3.76863e+01 ... -7.63701e+01  3.46391e+00
   1.00000e-04]
 [ 3.95852e+01  9.48854e+01 -3.76639e+01 ... -9.48854e+01  3.45005e+00
   2.00000e-04]
 ...
 [-3.47787e-05  5.43907e+01 -5.84241e-03 ... -5.43907e+01  1.53169e-02
   3.50820e+00]
 [-1.48180e-04  5.37585e+01 -5.70406e-03 ... -5.37585e+01  1.51716e-02
   3.50830e+00]
 [-2.59909e-04  5.312

[[ 3.96079e+01  7.63701e+01 -3.76863e+01 ... -7.63701e+01  3.46391e+00
   1.00000e-04]
 [ 3.95842e+01  9.48659e+01 -3.76629e+01 ... -9.48659e+01  3.44949e+00
   2.00000e-04]
 [ 3.95375e+01  1.12935e+02 -3.76168e+01 ... -1.12935e+02  3.42110e+00
   3.00000e-04]
 ...
 [-3.89339e-03  3.05236e+01  1.77479e-03 ... -3.05236e+01  6.53449e-03
   4.99980e+00]
 [-3.94128e-03  3.02638e+01  1.83674e-03 ... -3.02638e+01  6.46247e-03
   4.99990e+00]
 [-3.98818e-03  3.00028e+01  1.89764e-03 ... -3.00028e+01  6.39429e-03
   5.00000e+00]]
(50000, 7)
[[ 0.00000e+00  0.00000e+00  0.00000e+00 ...  0.00000e+00  0.00000e+00
   0.00000e+00]
 [ 3.96079e+01  7.63701e+01 -3.76863e+01 ... -7.63701e+01  3.46391e+00
   1.00000e-04]
 [ 3.95842e+01  9.48659e+01 -3.76629e+01 ... -9.48659e+01  3.44949e+00
   2.00000e-04]
 ...
 [-3.89339e-03  3.05236e+01  1.77479e-03 ... -3.05236e+01  6.53449e-03
   4.99980e+00]
 [-3.94128e-03  3.02638e+01  1.83674e-03 ... -3.02638e+01  6.46247e-03
   4.99990e+00]
 [-3.98818e-03  3.000